The SK model:
### Sherrington–Kirkpatrick (SK) Spin Glass Model

The Hamiltonian of the Sherrington–Kirkpatrick model is

$$
\mathcal{H}_N(\boldsymbol{\sigma})
= - \sum_{1 \le i < j \le N} J_{ij}\,\sigma_i \sigma_j
\;-\;
\sum_{i=1}^{N} h_i\,\sigma_i ,
$$

where the terms are defined as follows:

- $\sigma_i \in \{+1,-1\}$ are **Ising spins**, representing binary degrees of freedom.
- $J_{ij}$ are **quenched random couplings** between spins $i$ and $j$, drawn independently from a Gaussian distribution
  $$
  J_{ij} \sim \mathcal{N}\!\left(0,\frac{J^2}{N}\right),
  $$
  often with $J^2 = 1$.
- The scaling $1/N$ ensures a **well-defined thermodynamic limit**, so that the total energy is extensive (of order $N$).
- $h_i$ is an **external field** acting on spin $i$ (often taken as $h_i = h$ or $h_i = 0$).
- The first term encodes **frustrated interactions** due to random signs of $J_{ij}$.
- The second term biases spins toward alignment with the external field.

The SK model is a **mean-field spin glass**, meaning every spin interacts with every other spin with random strength.


### The Transformer architecture and its components

### High-level Thesis Vision

This thesis investigates a two-way bridge between spin glass theory and transformer models:

Forward direction (Goal I):
Use transformers as learned statistical models to approximate and simulate equilibrium properties of spin glasses.

Reverse direction (Goal II):
Use spin-glass concepts (overlaps, Gibbs measures, replica symmetry breaking, free energy) to develop new tools for interpretability and intervention in transformers.

The unifying idea is that both systems are large, high-dimensional, disordered systems whose macroscopic behavior is better described by distributions over states than by individual parameters.

Goal 1 : do MCMC sampling to generate the data which will train the transformer, see if attention is closest h_eff_i ( which is the effective field acting on spin i due to the configuration from other spins, How accurately does it predict ground state properties such as free Energy, magnetization etc )


Goal 2: Can Spin Glass ideas such as ultrametricity, Replica Symmetry Breaking and  cavity method, improve interpretability and intervenability in Transformers. 